<b><h1 style="text-align:center;">SynCoat Squad: Advancing Smart Coatings with Genetic Algorithm Optimization</h1>
</b>

In the growing paints industry, this model optimizes material formulations, considering properties like durability, self-healing, and UV resistance. It automates the process, improving product performance and **reducing R&D costs** by quickly finding the best formulations.

##Assumptions
* The properties of materials (e.g., durability, self-healing) are considered relative to one another, based on the values provided in the materials dictionary.
*  Material properties are constant and do not change with interactions in the formulation.
*  Material properties combine linearly without synergistic or antagonistic interactions.
*  Material properties are normalized to a scale of 0–1 for consistency.
*  The fitness function weights properties like durability and cost to reflect their importance.


## Workflow of the model
- **Input**: A dictionary of materials with properties such as durability, cost, UV resistance, etc.
- **Objective**: Find the best combination of materials that maximizes the performance of the coating while balancing cost and other trade-offs.
- **Process**:
 - Initialize a population of random formulations.
 -Use a fitness function to evaluate each formulation based on weighted material properties.
 -Select parent formulations, perform crossover (combining material proportions), and apply mutation (introducing small random changes).
 -Repeat for a set number of generations or until improvements stagnate.
 -Output the optimal formulation.






## Features
* **Mutation**: Introduces random changes to material proportions (±0.1) with a small probability to explore new solutions.
* **Threshold for Improvement**: Stops the algorithm early if no significant fitness improvement occurs (fitness_threshold = 0.01).
* **Stagnation Limit**: Ends the process after 5 generations without improvement to save resources.

## Workflow of the GA
*  **Initialization**: Start with random formulations (proportions of materials normalized to sum to 1).
*  **Fitness Evaluation**: Score each formulation based on properties like durability, cost, and miscibility.
*  **Selection and Evolution**: Use the genetic algorithm to iteratively improve formulations through selection, crossover, and mutation.
*  **Early Stopping**: Stop if the algorithm stagnates, outputting the best formulation found.

In [ ]:
import numpy as np
import random

random.seed(42) # to get same answer every time when not changes anything in the code
np.random.seed(42)

# Materials with updated properties
materials = {
    "Epoxy_Resin": {"durability": 0.8, "self_healing": 0.5, "cost": 1.2, "corrosion_resistance": 0.9,
                    "UV_resistance": 0.7, "hydrophobicity": 0.3, "antimicrobial": 0.4,
                    "adhesion_strength": 0.8, "miscibility": 0.9, "reaction_tendency": 0.2},
    "Polyurethane": {"durability": 0.9, "self_healing": 0.6, "cost": 1.0, "corrosion_resistance": 0.8,
                    "UV_resistance": 0.9, "hydrophobicity": 0.8, "antimicrobial": 0.5,
                    "adhesion_strength": 0.7, "miscibility": 0.9, "reaction_tendency": 0.3},
    "Silica_Nanoparticles": {"durability": 0.7, "self_healing": 0.4, "cost": 1.5, "corrosion_resistance": 1.0,
                            "UV_resistance": 0.8, "hydrophobicity": 0.5, "antimicrobial": 0.3,
                            "adhesion_strength": 0.6, "miscibility": 0.6, "reaction_tendency": 0.1},
    "Healing_Agent": {"durability": 0.5, "self_healing": 1.0, "cost": 2.0, "corrosion_resistance": 0.3,
                      "UV_resistance": 0.3, "hydrophobicity": 0.1, "antimicrobial": 0.6,
                      "adhesion_strength": 0.5, "miscibility": 0.4, "reaction_tendency": 0.7},
}

# Parameters for the genetic algorithm
population_size = 45
generations = 50
mutation_rate = 0.1
fitness_threshold = 0.01  #for early stopping
stagnation_limit = 5  # final limit to stop
no_improvement_count = 0

# Normalize material properties
def normalize_material_properties(materials):
    normalized_materials = {}
    for material, properties in materials.items():
        normalized_properties = {}
        for prop, value in properties.items():

            all_values = [m[prop] for m in materials.values()]
            min_val, max_val = min(all_values), max(all_values)

            # Normalize
            normalized_value = (value - min_val) / (max_val - min_val)
            normalized_properties[prop] = normalized_value
        normalized_materials[material] = normalized_properties
    return normalized_materials

# Apply normalization
materials = normalize_material_properties(materials)

# Fitness function to score formulations based on weighted objectives
def fitness(formulation):
    durability = sum(materials[mat]["durability"] * qty for mat, qty in formulation.items())
    self_healing = sum(materials[mat]["self_healing"] * qty for mat, qty in formulation.items())
    cost = sum(materials[mat]["cost"] * qty for mat, qty in formulation.items())
    corrosion_resistance = sum(materials[mat]["corrosion_resistance"] * qty for mat, qty in formulation.items())
    adhesion_strength = sum(materials[mat]["adhesion_strength"] * qty for mat, qty in formulation.items())
    miscibility = sum(materials[mat]["miscibility"] * qty for mat, qty in formulation.items())
    reaction_tendency = sum(materials[mat]["reaction_tendency"] * qty for mat, qty in formulation.items())

    #fitness function with weights
    return (0.8 * durability + 0.5 * self_healing + 0.6 * corrosion_resistance +
            0.7 * adhesion_strength + 0.5 * miscibility - 0.15 * cost -
            0.05 * reaction_tendency)

# Initialize a population of random formulations
def initialize_population():
    population = []
    for _ in range(population_size):
        formulation = {mat: random.uniform(0, 1) for mat in materials.keys()}

        # Normalize
        total = sum(formulation.values())
        for mat in formulation:
            formulation[mat] /= total

        population.append(formulation)
    return population

# Selection process based on fitness
def select_parents(population):
    fitness_scores = [fitness(formulation) for formulation in population]
    selected = np.random.choice(population, size=2, p=np.array(fitness_scores)/sum(fitness_scores))
    return selected

# Crossover function to combine genes from two parents
def crossover(parent1, parent2):
    child = {}
    for mat in materials.keys():
        child[mat] = (parent1[mat] + parent2[mat]) / 2

    # Normalize
    total = sum(child.values())
    for mat in child:
        child[mat] /= total

    return child

# Mutation function to introduce random changes
def mutate(formulation):
    for mat in formulation.keys():
        if random.random() < mutation_rate:
            formulation[mat] += random.uniform(-0.1, 0.1)

    # Normalize
    total = sum(formulation.values())
    for mat in formulation:
        formulation[mat] /= total

    return formulation

def genetic_algorithm():
    population = initialize_population()
    best_fitness = -float('inf')
    for gen in range(generations):
        new_population = []
        for _ in range(population_size // 2):
            parent1, parent2 = select_parents(population)
            child1, child2 = crossover(parent1, parent2), crossover(parent2, parent1)
            new_population.extend([mutate(child1), mutate(child2)])
        population = new_population

        # Get the best formulation from the population
        current_best_formulation = max(population, key=fitness)
        current_best_fitness = fitness(current_best_formulation)

        # Check for improvement in fitness
        if current_best_fitness - best_fitness < fitness_threshold:
            no_improvement_count += 1
        else:
            no_improvement_count = 0

        best_fitness = current_best_fitness

        # early stopping according to stagnation limit
        if no_improvement_count >= stagnation_limit:
            print(f"Early stopping at generation {gen + 1}. Best Fitness: {best_fitness:.4f}")
            break

        print(f"Generation {gen + 1}, Best Fitness: {best_fitness:.4f}, Best Formulation: {current_best_formulation}")

    return current_best_formulation

# Get optimal formulation from genetic algorithm
optimal_formulation = genetic_algorithm()
print("Optimal Formulation:", optimal_formulation)


Generation 1, Best Fitness: 2.1774, Best Formulation: {'Epoxy_Resin': 0.6775395753814552, 'Polyurethane': 0.20632453966064393, 'Silica_Nanoparticles': 0.05685102066774211, 'Healing_Agent': 0.05928486429015886}
Generation 2, Best Fitness: 1.9984, Best Formulation: {'Epoxy_Resin': 0.5164905415752193, 'Polyurethane': 0.21944096602733523, 'Silica_Nanoparticles': 0.1756986924826384, 'Healing_Agent': 0.08836979991480712}
Generation 3, Best Fitness: 1.8791, Best Formulation: {'Epoxy_Resin': 0.39414022835946644, 'Polyurethane': 0.2674443197936625, 'Silica_Nanoparticles': 0.20834613088669743, 'Healing_Agent': 0.1300693209601736}
Generation 4, Best Fitness: 1.8514, Best Formulation: {'Epoxy_Resin': 0.41385717004962613, 'Polyurethane': 0.23063668986436264, 'Silica_Nanoparticles': 0.21494206199390017, 'Healing_Agent': 0.14056407809211102}
Generation 5, Best Fitness: 1.8379, Best Formulation: {'Epoxy_Resin': 0.3434964474989623, 'Polyurethane': 0.30276122869739813, 'Silica_Nanoparticles': 0.19970397